# CMDPにおける違反なしの探索

参考
* [Learning Policies with Zero or Bounded Constraint Violation for Constrained MDPs](https://arxiv.org/abs/2106.02684)
* これの証明かなりややこしいかも．DOPEを見よう．

CMDPで制約違反をせずにリグレットを抑えるアルゴリズムを考えましょう．

表記
* 論文参照．また，[RL_CMDP_explore_exploit.ipynb](RL_CMDP_explore_exploit.ipynb)とだいたい同じ
* $\pi^0$は$V_1^{\pi^0}(\mu ; c, P)=c^0<\tau$を満たすときにStrictly safeといいます．

問題設定

面倒なので報酬とコストは既知とします．
次のCMDPを考えます．
$$
\max _\pi V_1^\pi(\mu ; r, P) \quad \text { s.t. } \quad V_1^\pi(\mu ; c, P) \leq \tau \text {. }
$$

## 違反なしのCMDP

エージェントはstrictly safeな$\pi^0$を知っているとします．
また，そのコスト$c^0:=V_1^{\pi^0}(\mu ; c, P)$を知っているとします．

アルゴリズムの性能を次のリグレットで測定します．
$$
\operatorname{Reg}(K ; r):=\sum_{k=1}^K\left(V_1^{\pi^*}(\mu ; r, P)-V_1^{\pi^k}(\mu ; r, P)\right)
$$

また，次のように違反しないことを確率$1-\delta$以上で保証したいとします．

$$
\mathbb{P}\left(V_1^{\pi^k}(\mu ; c, P) \leq \tau, \forall k \in[K]\right) \geq 1-\delta
$$

---

**アルゴリズムの準備**

基本的には報酬にOptimismを追加し，コストにPessimismを追加するだけです．
まず，

$$
\hat{P}_h^k\left(s^{\prime} \mid s, a\right):=\frac{\sum_{k^{\prime}=1}^{k-1} \mathbb{1}\left(S_h^{k^{\prime}}=s, A_h^{k^{\prime}}=a, S_{h+1}^{k^{\prime}}=s^{\prime}\right)}{N_h^k(s, a) \vee 1},
$$

とします．
また，ボーナスを

$$
\beta_h^k(s, a):=\sqrt{\frac{1}{N_h^k(s, a) \vee 1} Z}, \quad \text { where } Z:=\log \left(16|\mathcal{S}|^2|\mathcal{A}| H K / \delta\right)
$$

とします．
報酬を

$$\bar{r}_h^k(s, a):={r}_h(s, a)+\alpha_r \beta_h^k(s, a), \quad \forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]$$

とます．ここで$\alpha_r:=1+|\mathcal{S}| H+\frac{4 H(1+|\mathcal{S}| H)}{\tau-c^0}$です．

コストは
$$
\underline{\mathrm{c}}_h^k(s, a):={c}_h(s, a)+(1+H|\mathcal{S}|) \beta_h^k(s, a), \quad \forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]
$$
とします．

そして，方策を

$$
\Pi^k:= \begin{cases}\left\{\pi^0\right\} & \text { if } V_1^{\pi^0}\left(\mu ; \underline{\mathbf{c}}^k, \hat{P}^k\right) \geq\left(\tau+c^0\right) / 2 \\ \left\{\pi: V_1^\pi\left(\mu ; \underline{\mathbf{c}}^k, \hat{P}^k\right) \leq \tau\right\} & \text { otherwise. }\end{cases}
$$

として選択します．
つまり，$\pi^0$が$V_1^{\pi^0}\left(\mu ; \underline{\mathrm{c}}^k, \hat{P}^k\right)<\left(\tau+c^0\right) / 2$を満たすまで選択され続けます．
これは$\pi^0$が$\underline{\mathrm{c}}^k, \hat{P}^k$において安全である状況です．
$\tau \leq c^0$であり，$\pi^0$が訪れる範囲の精度がどんどん向上していくので，これはいつか満たされます．

これは$\left\{\pi: V_1^\pi\left(\mu ; \underline{\mathrm{c}}^k, \hat{P}^k\right) \leq \tau\right\}$は空でないことの十分条件です（最低でも$\pi^0$は含まれます）．

---

**アルゴリズム**

OptPress-LPアルゴリズムでは次を実行します：

1. for $k=1, 2, \dots, K$
2. $\hat{P}^k$を計算します
3. $\bar{r}^k, \underline{\mathrm{c}}^k$, and $\Pi^k$を計算します
4. $\pi^k \in \arg \max _{\pi \in \Pi^k} V_1^\pi\left(\mu ; \bar{r}^k, \hat{P}^k\right)$を計算します．
5. データを集めます

## リグレットバウンド

次が成立します：

OptPress-LPアルゴリズムは，確率$1 - \delta$以上で，次のリグレットを違反無しで達成する：

$$
\operatorname{Reg}^{\text {OPLP }}(K ; r)=\tilde{\mathcal{O}}\left(\frac{H^3}{\tau-c^0} \sqrt{|\mathcal{S}|^3|\mathcal{A}| K}+\frac{H^5|\mathcal{S}|^3|\mathcal{A}|}{\left(\tau-c^0\right)^2 \wedge\left(\tau-c^0\right)}\right) \text {, where } a \wedge b:=\min \{a, b\}
$$

このバウンドを証明していきましょう．

### Good Events

基本的に遷移の推定精度が一定以下に抑えられることをGood Eventにします．

---

**簡単なやつ**

まずは簡単なものから考えます．

$$
\begin{aligned}
E^P = \{
&\left|\hat{P}_h^k\left(s^{\prime} \mid s, a\right)-P_h\left(s^{\prime} \mid s, a\right)\right| \leq \beta_h^k(s, a)\\
&\left|\hat{P}_h^k\left(s^{\prime} \mid s, a\right)-P_h\left(s^{\prime} \mid s, a\right)\right| \leq \tilde{\beta}_h^k\left(s^{\prime} \mid s, a\right)
\;  \forall (k, h, s, a, s')\}
\end{aligned}
$$

ここで，$\tilde{\beta}_h^k\left(s^{\prime} \mid s, a\right):=\sqrt{\frac{2 P_h\left(s^{\prime} \mid s, a\right)}{N_h^k(s, a) \vee 1} Z}+\frac{Z}{3 N_h^k(s, a) \vee 1}$ and $Z:=\log \left(16|\mathcal{S}|^2|\mathcal{A}| H K / \delta\right)$としました．

これが高確率で起きることを示します．

そのために，それぞれの$(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]$について，$K$個の独立した次状態をgenerative modelからサンプルする状況を考えます：

$$
\left\{S^n(s, a, h)\right\}_{n=1}^K
$$

これに対して，$\hat{P}^n(\cdot \mid s, a, h)$を$n$個のサンプル

$$
\left\{S^i(s, a, h)\right\}_{i=1}^n
$$

を使って計算した経験遷移とします．


ここで，Failure Eventを
$$F_n^P :=\left\{\exists s, a, s^{\prime}, h:\left|P_h\left(s^{\prime} \mid s, a\right)-\hat{P}^n\left(s^{\prime} \mid s, a, h\right)\right| \geq \beta(n)\right\}$$
$$\tilde{F}_n^P:=\left\{\exists s, a, s^{\prime}, h:\left|P_h\left(s^{\prime} \mid s, a\right)-\hat{P}^n\left(s^{\prime} \mid s, a, h\right)\right| \geq \tilde{\beta}\left(n, P_h\left(s^{\prime} \mid s, a\right)\right)\right\}$$

とします．ここで$\beta(n):=\sqrt{\frac{1}{n \vee 1} Z}$かつ$\tilde{\beta}(n, p):=\sqrt{\frac{2 P\left(s^{\prime} \mid s, a\right)}{n \vee 1} Z}+\frac{Z}{3 n \vee 1}$です．

$E^P = \left(\bigcup_{n=1}^K F_n^P \cup \tilde{F}_n^P \right)^C$であり，

* Hoeffdingより$\mathbb{P}\left(\bigcup_{n=1}^K F_n^P\right) \leq|\mathcal{S}|^2|\mathcal{A}| H K \frac{\delta}{16|\mathcal{S}|^2|\mathcal{A}| H K} \leq \delta^{\prime}$
* Bernstein（UCBVI参照）より$\mathbb{P}\left(\bigcup_{n=1}^K \tilde{F}_n^P\right) \leq|\mathcal{S}|^2|\mathcal{A}| H K \frac{\delta}{16|\mathcal{S}|^2|\mathcal{A}| H K} \leq \delta^{\prime}$

なので，簡単にHigh probabilityで起きることが示せます．ここで$\delta / 16 \leq \delta|\mathcal{S}| / 4(1+|\mathcal{S}|)=: \delta^{\prime}$としました．あとはUnion boundを取るだけです．

---

**ややこしいやつ**

後で便利なので，次の補題を示します．（TODO: これどこで使ってるの？）

---

**コメント**：これ微妙か？[Learning Adversarial MDPs with Bandit Feedback and Unknown Transition](https://arxiv.org/abs/1912.01192)のLemma 10を使ったほうがわかりやすいかも．つまり，$\mathcal{E}^k_h(s, a)$をステップ$h$で$(s, a)$を訪れるイベントとしてごちゃごちゃする．

まず，フィルトレーション
$\left\{\mathcal{F}_k\right\}_{k \geq 0}$
を次のように定義します．

$\mathcal{F}_0$はtrivialな$\sigma$代数とします．それぞれの$k \in [K]$について，
$$\mathcal{F}_k=\sigma\left(\left(S_h^{k^{\prime}}, A_h^{k^{\prime}}\right)_{h \in[H], k^{\prime} \in[k]}\right)$$
とします．

このとき，方策の系列$\left\{\pi^k\right\}_{k \in[K]}$はFiltration $\left\{\mathcal{F}_k\right\}_{k \geq 0}$についてのpredictable processです．

ここで，$\mathcal{G}_k \in \mathcal{F}_{k-1}$ for each $k \in[K]$なるイベントの集合$\mathcal{G}_{1:K}$について，次が任意の$K'\leq K$について，確率$1-\delta$以上で成立します：

$$
\begin{aligned}
& \sum_{k=1}^{K^{\prime}} \sum_{h=1}^H \sum_{s, a} \frac{\mathbb{1}\left(\mathcal{G}_k\right) q^{\pi^k}(s, a, h)}{N_h^k(s, a) \vee 1} \leq 4 H|\mathcal{S}\|\mathcal{A}|+2 H| \mathcal{S}\| \mathcal{A}| \ln K_{\mathcal{G}}^{\prime}+4 \ln \frac{2 H K}{\delta} \\
& \sum_{k=1}^{K^{\prime}} \sum_{h=1}^H \sum_{s, a} \frac{\mathbb{1}\left(\mathcal{G}_k\right) q^{\pi^k}(s, a, h)}{\sqrt{N_h^k(s, a) \vee 1}} \leq 6 H|\mathcal{S}||\mathcal{A}|+2 H \sqrt{|\mathcal{S}||\mathcal{A}| K_{\mathcal{G}}^{\prime}}+2 H|\mathcal{S}||\mathcal{A}| \ln K_{\mathcal{G}}^{\prime}+5 \ln \frac{2 H K}{\delta}
\end{aligned}
$$

ここで，$K_{\mathcal{G}}^{\prime}:=\sum_{k=1}^{K^{\prime}} \mathbb{1}\left(\mathcal{G}_k\right)$，かつ$q^{\pi^k}$は方策のOccupancy measureです．
つまり，$q^{\pi^k}(s, a, h)=\mathbb{E}_{\mu, P, \pi^k}\left[\mathbb{1}\left(S_h^k=s, A_h^k=a\right) \mid \mathcal{F}_{k-1}\right]$です．

---



さて，predictableなイベントの系列
$\mathcal{G}_{1: K}$, i.e., $\mathcal{G}_k \in \mathcal{F}_{k-1}, \forall k \in[K]$,
について，次のイベントを定義します．

$$
\begin{aligned}
& \mathcal{E}_{\mathcal{G}}(\delta):=\left\{\forall K^{\prime} \in[K], \sum_{k=1}^{K^{\prime}} \sum_{h=1}^H \sum_{s, a} \frac{\mathbb{1}\left(\mathcal{G}_k\right) q^{\pi^k}(s, a, h)}{N_h^k(s, a) \vee 1} \leq 4 H|\mathcal{S}||\mathcal{A}|+2 H|\mathcal{S}||\mathcal{A}| \ln K_{\mathcal{G}}^{\prime}+4 \ln \frac{2 H K}{\delta},\right. \\
& \left.\sum_{k=1}^{K^{\prime}} \sum_{h=1}^H \sum_{s, a} \frac{\mathbb{1}\left(\mathcal{G}_k\right) q^{\pi^k}(s, a, h)}{\sqrt{N_h^k(s, a) \vee 1}} \leq 6 H|\mathcal{S}||\mathcal{A}|+2 H \sqrt{|\mathcal{S}||\mathcal{A}| K_{\mathcal{G}}^{\prime}}+2 H|\mathcal{S}||\mathcal{A}| \ln K_{\mathcal{G}}^{\prime}+5 \ln \frac{2 H K}{\delta}\right\},
\end{aligned}
$$

ここで，$K_{\mathcal{G}}^{\prime}:=\sum_{k=1}^{K^{\prime}} \mathbb{1}\left(\mathcal{G}_k\right)$，かつ$q^{\pi^k}$は方策のOccupancy measureです．
つまり，$q^{\pi^k}(s, a, h)=\mathbb{E}_{\mu, P, \pi^k}\left[\mathbb{1}\left(S_h^k=s, A_h^k=a\right) \mid \mathcal{F}_{k-1}\right]$です．

このとき，イベント
$$
\mathcal{E}_{\Omega}(\delta / 4) \cap \mathcal{E}_0(\delta / 4),
$$

は，上の補題を使うと確率$1-\delta / 2$以上で生じます．
ここで，
$\mathcal{G}_{1: K}^{\prime}=\left\{V_1^{\pi^0}\left(\mu ; \underline{\mathrm{c}}^k, \hat{P}^k\right) \geq \frac{\tau+c^0}{2}\right\}_{k \in[K]}$
について，
$\mathcal{E}_0(\delta)$ は$\mathcal{E}_{\mathcal{G}^{\prime}}(\delta)$の略記です．


よって，次のGood eventは確率$1-\delta$以上で生じます：

$$
\begin{aligned}
\mathcal{E}:=\{ & \forall k \in[K], \forall h \in[H], \forall s \in \mathcal{S}, \forall a \in \mathcal{A}, \\
& \left|\hat{P}_h^k\left(s^{\prime} \mid s, a\right)-P_h\left(s^{\prime} \mid s, a\right)\right| \leq \beta_h^k(s, a), \forall s^{\prime} \in \mathcal{S} \\
& \left.\left|\hat{P}_h^k\left(s^{\prime} \mid s, a\right)-P_h\left(s^{\prime} \mid s, a\right)\right| \leq \tilde{\beta}_h^k\left(s^{\prime} \mid s, a\right), \forall s^{\prime} \in \mathcal{S}\right\} \cap \mathcal{E}_{\Omega}(\delta / 4) \cap \mathcal{E}_0(\delta / 4),
\end{aligned}
$$


**Good Eventの証明**

次のFailure Eventを定義します．

$$
\begin{aligned}
F_n^P & :=\left\{\exists s, a, s^{\prime}, h:\left|P_h\left(s^{\prime} \mid s, a\right)-\hat{P}^n\left(s^{\prime} \mid s, a, h\right)\right| \geq \beta(n)\right\}, \\
\tilde{F}_n^P & :=\left\{\exists s, a, s^{\prime}, h:\left|P_h\left(s^{\prime} \mid s, a\right)-\hat{P}^n\left(s^{\prime} \mid s, a, h\right)\right| \geq \tilde{\beta}\left(n, P_h\left(s^{\prime} \mid s, a\right)\right)\right\},
\end{aligned}
$$


まず，リグレットを分解します：

$$
\begin{aligned}
\operatorname{Reg}^{\mathbf{O P L P}}(K ; r) & =\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|=1\right)\left(V_1^{\pi^*}(\mu ; r, P)-V_1^{\pi^0}(\mu ; r, P)\right) \\
& +\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|>1\right)\left(V_1^{\pi^*}(\mu ; r, P)-V_1^{\pi^k}\left(\mu ; \bar{r}^k, \hat{P}^k\right)\right) \\
& +\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|>1\right)\left(V_1^{\pi^k}\left(\mu ; \bar{r}^k, \hat{P}^k\right)-V_1^{\pi^k}(\mu ; r, P)\right)
\end{aligned}
$$

最初の項はつまり$\pi^0$しか選択できない場合ですね．
１つ目の項は次でバウンドできます：

---

**補題**

確率$(1-\delta)$以上で，

$$
\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|=1\right) \leq C^{\prime}
$$

が成立し，ここで，$C'=\tilde{\mathcal{O}}\left(H^4|\mathcal{S}|^3|\mathcal{A}| /\left(\left(\tau-c^0\right)^2 \wedge\left(\tau-c^0\right)\right)\right)$です．



### Good Events

predictableなイベントの系列
$\mathcal{G}_{1: K}$, i.e., $\mathcal{G}_k \in \mathcal{F}_{k-1}, \forall k \in[K]$,
について，次のイベントを定義します．

$$
\begin{aligned}
& \mathcal{E}_{\mathcal{G}}(\delta):=\left\{\forall K^{\prime} \in[K], \sum_{k=1}^{K^{\prime}} \sum_{h=1}^H \sum_{s, a} \frac{\mathbb{1}\left(\mathcal{G}_k\right) q^{\pi^k}(s, a, h)}{N_h^k(s, a) \vee 1} \leq 4 H|\mathcal{S}||\mathcal{A}|+2 H|\mathcal{S}||\mathcal{A}| \ln K_{\mathcal{G}}^{\prime}+4 \ln \frac{2 H K}{\delta},\right. \\
& \left.\sum_{k=1}^{K^{\prime}} \sum_{h=1}^H \sum_{s, a} \frac{\mathbb{1}\left(\mathcal{G}_k\right) q^{\pi^k}(s, a, h)}{\sqrt{N_h^k(s, a) \vee 1}} \leq 6 H|\mathcal{S}||\mathcal{A}|+2 H \sqrt{|\mathcal{S}||\mathcal{A}| K_{\mathcal{G}}^{\prime}}+2 H|\mathcal{S}||\mathcal{A}| \ln K_{\mathcal{G}}^{\prime}+5 \ln \frac{2 H K}{\delta}\right\},
\end{aligned}
$$

ここで，$K_{\mathcal{G}}^{\prime}:=\sum_{k=1}^{K^{\prime}} \mathbb{1}\left(\mathcal{G}_k\right)$，かつ$q^{\pi^k}$は方策のOccupancy measureです．
つまり，$q^{\pi^k}(s, a, h)=\mathbb{E}_{\mu, P, \pi^k}\left[\mathbb{1}\left(S_h^k=s, A_h^k=a\right) \mid \mathcal{F}_{k-1}\right]$です．

Good Eventを次で定義します：

$$
\begin{aligned}
\mathcal{E}:=\{ & \forall k \in[K], \forall h \in[H], \forall s \in \mathcal{S}, \forall a \in \mathcal{A}, \\
& \left|\hat{P}_h^k\left(s^{\prime} \mid s, a\right)-P_h\left(s^{\prime} \mid s, a\right)\right| \leq \beta_h^k(s, a), \forall s^{\prime} \in \mathcal{S} \\
& \left.\left|\hat{P}_h^k\left(s^{\prime} \mid s, a\right)-P_h\left(s^{\prime} \mid s, a\right)\right| \leq \tilde{\beta}_h^k\left(s^{\prime} \mid s, a\right), \forall s^{\prime} \in \mathcal{S}\right\} \cap \mathcal{E}_{\Omega}(\delta / 4) \cap \mathcal{E}_0(\delta / 4),
\end{aligned}
$$

$\tilde{\beta}_h^k\left(s^{\prime} \mid s, a\right):=\sqrt{\frac{2 P\left(s^{\prime} \mid s, a\right)}{N_h^k(s, a) \vee 1} Z}+\frac{Z}{3 N_h^k(s, a) \vee 1}$ and $Z:=\log \left(16|\mathcal{S}|^2|\mathcal{A}| H K / \delta\right)$としました

このとき，このGood Eventが確率$1-\delta$以上で生じることを示しましょう．

